# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Load Data
city_df= pd.read_csv('../output_data/weather_data.csv')
city_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kasama,-10.21,31.18,18.10,95,71,1.87,ZM,1607103773
1,1,meulaboh,4.14,96.13,25.28,87,100,2.18,ID,1607103773
2,2,pozoblanco,38.38,-4.85,9.00,76,20,3.60,ES,1607103774
3,3,taltal,-25.40,-70.48,19.06,58,23,3.31,CL,1607103774
4,4,pella,59.78,30.82,0.00,94,90,5.00,RU,1607103774
...,...,...,...,...,...,...,...,...,...,...
56,56,laranjal paulista,-23.05,-47.84,33.33,41,43,0.45,BR,1607103780
57,57,asau,46.43,26.40,3.33,40,24,1.33,RO,1607103780
58,58,smolensk,55.00,33.00,-2.60,96,94,4.78,RU,1607103781
59,59,puri,19.80,85.85,20.00,88,2,2.07,IN,1607103781


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
location=city_df[['Lat','Lng']]
humidity=city_df['Humidity']

In [5]:
# Create a Heatmap layer
%matplotlib inline
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                 dissipating=False, max_intensity=500,
                                 point_radius = 6)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
new_df=city_df.loc[(city_df['Max Temp']<26.66)&(city_df['Max Temp']>21.11)&(city_df[' Wind Speed']<10)&(city_df['Cloudiness']==0)]
new_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,calama,-22.47,-68.93,23.00,17,0,9.80,CL,1607103779
50,50,ormara,25.21,64.64,25.65,62,0,4.12,PK,1607103780


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#creating a new data frame
hotel_df=new_df[['City','Lat','Lng','Country']]
hotel_df

,City,Lat,Lng,Country
45,calama,-22.47,-68.93,CL
50,ormara,25.21,64.64,PK


In [9]:
#Adding a coloum on data frame 
hotel_df['Hotel Name']=''
hotel_df

C:\Users\asna_\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Lat,Lng,Country,Hotel Name
45,calama,-22.47,-68.93,CL,
50,ormara,25.21,64.64,PK,


In [10]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

C:\Users\asna_\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.


,City,Lat,Lng,Country,Hotel Name
45,calama,-22.47,-68.93,CL,Park Hotel Calama
50,ormara,25.21,64.64,PK,


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))